<a href="https://colab.research.google.com/github/QaziSaim/Post-Discharge-Medical-AI-Assistant-POC-/blob/main/Medical_MultiAgent_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU crewai
!pip install crewai_tools
!pip install -qU langchain-groq
!pip install sentence-transformers
